In [73]:
import pandas as pd
from fpl import Loader, Player

## Expected Points Model

Using the framework of [General Model Theory](https://en.wikipedia.org/wiki/Model)

##### Mapping - what phenomena are we trying to model?
Given that a player plays a particular fixture (greater than zero minutes played), we want to model how many points the player will score.

##### Reduction - what simplifying assumptions will we make?
The simplifying assumptions we will make are as follows:
- Points scored depends only on fixture difficulty, form and points per game (again, only games where a player plays more than zero minutes)
- There will be no attempt to estimate minutes of each player or likelihood of playing; the effect of minutes will come indirectly through points per game
- There will be no attempt to use other useful statistics from other data sources such as betting odds, opta data, xG etc

##### Pragmatism - when can we use the model from a practical point of view?
We can use this model if, for each fixture each player plays, we can calculate:
1. Their points per game so far that season, prior to the fixture.
2. Their form so far than season prior to the fixture.
3. An indication of the difficulty of a particular fixture.

This should be relatively simple, therefore we can create an `ExpectedPointsCalculator` from such a model and plug it into our `Optimizer`.

In [16]:
Loader.find_matching_players("Alexander-arnold")

[(Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=74),
  'Trent Alexander-Arnold'),
 (Player(element=642, name='André', position=3, club=20, cost=50),
  'André Trindade da Costa Neto')]

In [12]:
Loader.get_player_historical_info_for_gameweek(328, 24)

[{'element': 328,
  'fixture': 232,
  'opponent_team': 3,
  'total_points': 16,
  'was_home': False,
  'kickoff_time': '2025-02-01T15:00:00Z',
  'team_h_score': 0,
  'team_a_score': 2,
  'round': 24,
  'modified': False,
  'minutes': 87,
  'goals_scored': 2,
  'assists': 0,
  'clean_sheets': 1,
  'goals_conceded': 0,
  'own_goals': 0,
  'penalties_saved': 0,
  'penalties_missed': 0,
  'yellow_cards': 0,
  'red_cards': 0,
  'saves': 0,
  'bonus': 3,
  'bps': 59,
  'influence': '76.6',
  'creativity': '31.6',
  'threat': '60.0',
  'ict_index': '16.8',
  'starts': 1,
  'expected_goals': '0.88',
  'expected_assists': '0.39',
  'expected_goal_involvements': '1.27',
  'expected_goals_conceded': '1.53',
  'mng_win': 0,
  'mng_draw': 0,
  'mng_loss': 0,
  'mng_underdog_win': 0,
  'mng_underdog_draw': 0,
  'mng_clean_sheets': 0,
  'mng_goals_scored': 0,
  'value': 137,
  'transfers_balance': 54460,
  'selected': 8039715,
  'transfers_in': 65802,
  'transfers_out': 11342},
 {'element': 328,
  'f

In [74]:
def kickoff_time_in_last_30_days(kickoff_time: str) -> bool:
    as_of_ts = pd.Timestamp.now("UTC")
    thirty_days_ago = as_of_ts - pd.Timedelta(days=30)
    # Convert the kickoff_time string to a pandas Timestamp
    kickoff_time_ts = pd.Timestamp(kickoff_time, tz='UTC')
    return thirty_days_ago <= kickoff_time_ts

In [86]:
def compute_points_per_game(player_id: int) -> float:
    historical_fixtures = [info for i in range(1, Loader.get_next_gameweek()) for info in Loader.get_player_historical_info_for_gameweek(player_id, i)]
    points_array = [fixture["total_points"]for fixture in historical_fixtures if fixture["minutes"] > 0]
    return round(sum(points_array) / len(points_array), 1) if len(points_array) else 0.0

def compute_form(player_id: int) -> float:
    historical_fixtures = [info for i in range(1, Loader.get_next_gameweek()) for info in Loader.get_player_historical_info_for_gameweek(player_id, i)]
    points_array = [fixture["total_points"]for fixture in historical_fixtures if kickoff_time_in_last_30_days(fixture["kickoff_time"])]
    return round(sum(points_array) / len(points_array), 1) if len(points_array) else 0.0

In [104]:
for i in range(1, 772):
    computed_form_manual = compute_form(i)
    computed_form_official = Loader.get_player_basic_info(i)['form']
    computed_points_per_game_manual = compute_points_per_game(i)
    computed_points_per_game_official = Loader.get_player_basic_info(i)['points_per_game']
    computed_form_match = abs(computed_form_manual - float(computed_form_official)) < 0.1
    computed_points_per_game_match = abs(computed_points_per_game_manual - float(computed_points_per_game_official)) < 0.1
    print(f"player_id: {i}, expected == actual: {computed_form_match * computed_points_per_game_match}")

player_id: 1, expected == actual: 1
player_id: 2, expected == actual: 1
player_id: 3, expected == actual: 1
player_id: 4, expected == actual: 1
player_id: 5, expected == actual: 1
player_id: 6, expected == actual: 1
player_id: 7, expected == actual: 1
player_id: 8, expected == actual: 1
player_id: 9, expected == actual: 1
player_id: 10, expected == actual: 1
player_id: 11, expected == actual: 1
player_id: 12, expected == actual: 1
player_id: 13, expected == actual: 1
player_id: 14, expected == actual: 1
player_id: 15, expected == actual: 1
player_id: 16, expected == actual: 1
player_id: 17, expected == actual: 1
player_id: 18, expected == actual: 1
player_id: 19, expected == actual: 1
player_id: 20, expected == actual: 1
player_id: 21, expected == actual: 1
player_id: 22, expected == actual: 1
player_id: 23, expected == actual: 1
player_id: 24, expected == actual: 1
player_id: 25, expected == actual: 1
player_id: 26, expected == actual: 1
player_id: 27, expected == actual: 1
player_id:

player_id: 226, expected == actual: 1
player_id: 227, expected == actual: 1
player_id: 228, expected == actual: 1
player_id: 229, expected == actual: 1
player_id: 230, expected == actual: 1
player_id: 231, expected == actual: 1
player_id: 232, expected == actual: 1
player_id: 233, expected == actual: 1
player_id: 234, expected == actual: 1
player_id: 235, expected == actual: 1
player_id: 236, expected == actual: 1
player_id: 237, expected == actual: 1
player_id: 238, expected == actual: 1
player_id: 239, expected == actual: 1
player_id: 240, expected == actual: 1
player_id: 241, expected == actual: 1
player_id: 242, expected == actual: 1
player_id: 243, expected == actual: 1
player_id: 244, expected == actual: 1
player_id: 245, expected == actual: 1
player_id: 246, expected == actual: 1
player_id: 247, expected == actual: 1
player_id: 248, expected == actual: 1
player_id: 249, expected == actual: 1
player_id: 250, expected == actual: 1
player_id: 251, expected == actual: 1
player_id: 2

player_id: 445, expected == actual: 1
player_id: 446, expected == actual: 1
player_id: 447, expected == actual: 1
player_id: 448, expected == actual: 1
player_id: 449, expected == actual: 1
player_id: 450, expected == actual: 1
player_id: 451, expected == actual: 1
player_id: 452, expected == actual: 1
player_id: 453, expected == actual: 1
player_id: 454, expected == actual: 1
player_id: 455, expected == actual: 1
player_id: 456, expected == actual: 1
player_id: 457, expected == actual: 1
player_id: 458, expected == actual: 1
player_id: 459, expected == actual: 1
player_id: 460, expected == actual: 1
player_id: 461, expected == actual: 1
player_id: 462, expected == actual: 1
player_id: 463, expected == actual: 1
player_id: 464, expected == actual: 1
player_id: 465, expected == actual: 1
player_id: 466, expected == actual: 1
player_id: 467, expected == actual: 1
player_id: 468, expected == actual: 1
player_id: 469, expected == actual: 1
player_id: 470, expected == actual: 1
player_id: 4

player_id: 665, expected == actual: 1
player_id: 666, expected == actual: 1
player_id: 667, expected == actual: 1
player_id: 668, expected == actual: 1
player_id: 669, expected == actual: 1
player_id: 670, expected == actual: 1
player_id: 671, expected == actual: 1
player_id: 672, expected == actual: 1
player_id: 673, expected == actual: 1
player_id: 674, expected == actual: 1
player_id: 675, expected == actual: 1
player_id: 676, expected == actual: 1
player_id: 677, expected == actual: 1
player_id: 678, expected == actual: 1
player_id: 679, expected == actual: 1
player_id: 680, expected == actual: 1
player_id: 681, expected == actual: 1
player_id: 682, expected == actual: 1
player_id: 683, expected == actual: 1
player_id: 684, expected == actual: 1
player_id: 685, expected == actual: 1
player_id: 686, expected == actual: 1
player_id: 687, expected == actual: 1
player_id: 688, expected == actual: 1
player_id: 689, expected == actual: 1
player_id: 690, expected == actual: 1
player_id: 6

In [62]:
Loader.get_player_basic_info(2)

{'can_transact': True,
 'can_select': True,
 'chance_of_playing_next_round': 0,
 'chance_of_playing_this_round': 0,
 'code': 205651,
 'cost_change_event': 0,
 'cost_change_event_fall': 0,
 'cost_change_start': -4,
 'cost_change_start_fall': 4,
 'dreamteam_count': 2,
 'element_type': 4,
 'ep_next': '0.0',
 'ep_this': '0.0',
 'event_points': 0,
 'first_name': 'Gabriel',
 'form': '0.0',
 'id': 2,
 'in_dreamteam': False,
 'news': 'Knee injury - Unknown return date',
 'news_added': '2025-01-12T22:00:07.802845Z',
 'now_cost': 66,
 'photo': '205651.jpg',
 'points_per_game': '2.5',
 'removed': False,
 'second_name': 'Fernando de Jesus',
 'selected_by_percent': '1.2',
 'special': False,
 'squad_number': None,
 'status': 'i',
 'team': 1,
 'team_code': 3,
 'total_points': 42,
 'transfers_in': 1237537,
 'transfers_in_event': 106,
 'transfers_out': 1387552,
 'transfers_out_event': 9066,
 'value_form': '0.0',
 'value_season': '6.4',
 'web_name': 'G.Jesus',
 'region': 30,
 'team_join_date': '2022-07-